# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234954668343                   -0.50    7.0
  2   -7.250113563653       -1.82       -1.40    1.0
  3   -7.251267496764       -2.94       -2.01    4.0
  4   -7.251167033651   +   -4.00       -2.04    4.0
  5   -7.251320083079       -3.82       -2.53    3.0
  6   -7.251337107183       -4.77       -3.00    3.0
  7   -7.251338784696       -5.78       -4.17    2.0
  8   -7.251338794557       -8.01       -4.25    4.0
  9   -7.251338798422       -8.41       -4.80    3.0
 10   -7.251338798620       -9.70       -5.10    1.0
 11   -7.251338798699      -10.10       -5.68    3.0
 12   -7.251338798704      -11.29       -6.09    3.0
 13   -7.251338798704      -12.27       -6.51    2.0
 14   -7.251338798705      -13.29       -6.96    3.0
 15   -7.251338798705      -13.71       -7.44    2.0
 16   -7.251338798705   +  -15.05       -8.00    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.059012153375183084
[ Info: Arnoldi iteration step 2: normres = 0.5827627495098956
[ Info: Arnoldi iteration step 3: normres = 0.896046307412168
[ Info: Arnoldi iteration step 4: normres = 0.29924877404469064
[ Info: Arnoldi iteration step 5: normres = 0.25992044051359464
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.07e-02, 3.43e-02, 1.66e-01, 1.95e-01, 1.65e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5248878563652644
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (8.05e-03, 4.30e-02, 4.99e-01, 1.19e-01, 9.16e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0664222859478326
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.04e-04, 6.32e-03, 3.21e-02, 2.51e-02, 4.45e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09434666662643236
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.21e-